In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

In [ ]:
results_dir = "../results"
all_csv = sorted([f for f in os.listdir(results_dir) if f.endswith(".csv")])

In [ ]:
path = f"{results_dir}/{all_csv[-1]}"  # latest

In [ ]:
df = pd.read_csv(path, sep=";", comment="#")

In [ ]:
df["scheduled"] = df["deadline_misses"] == 0

In [ ]:
df.head()

In [ ]:
# Define bin edges and labels
bin_edges = np.arange(0, 1.1, 0.1)  # [0.0, 0.05, 0.10, ..., 1.0]
bin_labels = (bin_edges[:-1] + bin_edges[1:]) / 2  # label by bin centers

# Bin the utilization
df["util_bin"] = pd.cut(
    df["task_system_utilization"],
    bins=bin_edges,
    labels=bin_labels,
    include_lowest=True
)

# Group and count how many tasksets were scheduled per (bin, scheduler)
schedulability = (
    df[df["scheduled"] == True]
    .groupby(["util_bin", "scheduler"])
    .size()
    .reset_index(name="scheduled_count")
)

# Also compute total number of tasksets per bin (for horizontal reference line)
total_per_bin = df.groupby("util_bin").size()

# Plot
plt.figure(figsize=(10, 6))
sns.lineplot(
    data=schedulability,
    x="util_bin",
    y="scheduled_count",
    hue="scheduler",
    marker="o"
)

# Add horizontal lines showing the total number of tasksets per bin
for bin_center, total in total_per_bin.items():
    plt.axhline(y=total, color="gray", linestyle="--", linewidth=0.5)

# Force y-axis to start at 0
plt.ylim(bottom=0)

plt.title("Schedulability vs. Task System Utilization (Binned)")
plt.xlabel("Task System Utilization (Binned)")
plt.ylabel("# Scheduled Tasksets")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
df